In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
relationship = pd.read_csv('relationship_201203.csv',encoding = 'utf-8')
relationship = relationship.drop('Unnamed: 0',axis = 1)
relationship

,:START_ID,:TYPE,:END_ID,inverse
0,362633,birthPlace,5533231,direct
1,362633,birthPlace,1056076,direct
2,362633,deathPlace,5348598,direct
3,362633,deathPlace,3773633,direct
4,1324976,birthPlace,3429596,direct
...,...,...,...,...
26377579,1862626,type,4326933,directed
26377580,1862626,type,193423,directed
26377581,5175169,type,4635056,directed
26377582,1862626,type,470273,directed


In [3]:
entity = pd.read_csv('entity_201203.csv',encoding = 'utf-8')
entity = entity.drop('Unnamed: 0',axis = 1)
entity

,entityName,entityId:ID,entityProperty,entityType,:LABEL,entityImportance
0,"Iraqi parliamentary election, September 1954",0,"title: Iraqi parliamentary election, Septemb...",Election,Event,8.734346e+00
1,New Road Team,1,"name: New Road, name: New Road Team, nick: N...",SoccerClub,Organisation,1.066392e+01
2,Here Comes Trouble (Bad Company album),2,"name: Here Comes Trouble, releaseDate: 1992-0...",Album,Work,2.728453e+03
3,Canna glauca,3,"synonym: * Canna angustifolia (L.), synonym: ...",Plant,Species,1.519769e+00
4,Povla Frijsh,4,"name: Povla Frijsh, surname: Frijsh, givenNa...",Person,Person,1.612313e-01
...,...,...,...,...,...,...
4691086,German submarine U-133 (1941),5822928,"name: U-133, orderDate: 1939-08-07, layingDo...",Ship,Transportation,9.038188e-01
4691087,Rengarajan Jaiprakash,5822929,"name: Jaiprakash Rengarajan, description: Ind...",Person,Person,9.778662e+01
4691088,Jean d'Eaubonne,5822930,"name: Jean d Eaubonne, surname: Eaubonne, gi...",Person,Person,1.612313e-01
4691089,"Jack Price (footballer, born 1877) 2",5822931,"years: 1897, numberOfMatches: 1, numberOfGoa...",CareerStation,TimePeriod,6.397571e-01


In [4]:
entity[entity['entityImportance']>2290108]

,entityName,entityId:ID,entityProperty,entityType,:LABEL,entityImportance


In [5]:
entity[entity['entityName']=='China']

,entityName,entityId:ID,entityProperty,entityType,:LABEL,entityImportance
646255,China,802462,"longName: Peoples Republic of China, name: th...",Country,Place,18369.387539


# Test Demo

In [10]:
test_array = [[0,1],[0,2],[0,3],[1,0],[1,3],[2,3],[3,1]]
df = pd.DataFrame(test_array, columns = ['x', 'y'])
df

,x,y
0,0,1
1,0,2
2,0,3
3,1,0
4,1,3
5,2,3
6,3,1


In [11]:
# 另种意义上的邻接矩阵
lists_test = []
for i in range(4):
    temp_series = df[df.x==i]
    if len(temp_series)==0:
        lists_test.append([])
    else:
        temp_list = []
        for j in range(len(temp_series)):
            temp_list.append([temp_series.y.values[j],1])
        lists_test.append(temp_list)
lists_test

[[[1, 1], [2, 1], [3, 1]], [[0, 1], [3, 1]], [[3, 1]], [[1, 1]]]

In [12]:
# 计算每个节点的出边和S_test
S_test = df['x'].value_counts()
S_test

0    3
1    2
3    1
2    1
Name: x, dtype: int64

In [13]:
S_test[0]

3

In [14]:
# 另种意义上的转移概率矩阵 
for i in range(4):
    for j in range(len(lists_test[i])):
        if i in S_test.index:
            lists_test[i][j][1] = 1/S_test[i]
lists_test

[[[1, 0.3333333333333333], [2, 0.3333333333333333], [3, 0.3333333333333333]],
 [[0, 0.5], [3, 0.5]],
 [[3, 1.0]],
 [[1, 1.0]]]

In [15]:
page_rank_list(lists_test)

[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]
[0.25 0.25 0.25 0.25]


array([0.25, 0.25, 0.25, 0.25])

In [28]:
# 计算importance
def page_rank_df(lists_test, threshold = 10e-3, beta = 0.8):
    N = 4
    r_old = np.array([0 for i in range(N)])
    r_new = np.array([(1/N) for i in range(N)])
    while np.sum(np.abs(r_old - r_new)) > threshold:
        r_old = r_new
        dot_result = []
        for i in range(N): #x
            if len(lists_test[i])==0:
                dot_result.append(0)
            else:
                temp_result = 0
                for j in range(len(lists_test[i])):
                    temp_result+=lists_test[i][j][1]*r_new[lists_test[i][j][0]]
                dot_result.append(temp_result)
            print('dot_result:', dot_result)
        r_new = beta * np.array(dot_result)
        print(r_new)
        S = np.sum(r_new)
        r_new += (1-S)/N
        print('after add', r_new)
    return r_new

In [29]:
page_rank_df(lists_test)

dot_result: [0.25]
dot_result: [0.25, 0.25]
dot_result: [0.25, 0.25, 0.25]
dot_result: [0.25, 0.25, 0.25, 0.25]
[0.2 0.2 0.2 0.2]
after add [0.25 0.25 0.25 0.25]


array([0.25, 0.25, 0.25, 0.25])

# Revised Test Demo

In [3]:
test_array = [[0,1],[0,2],[0,3],[1,0],[1,3],[2,3],[3,1]]
df = pd.DataFrame(test_array, columns = ['x', 'y'])
df

,x,y
0,0,1
1,0,2
2,0,3
3,1,0
4,1,3
5,2,3
6,3,1


In [4]:
# 另种意义上的邻接矩阵
lists_test = []
for i in range(4):
    temp_series = df[df.y==i]
    if len(temp_series)==0:
        lists_test.append([])
    else:
        temp_list = []
        for j in range(len(temp_series)):
            temp_list.append([temp_series.x.values[j],1])
        lists_test.append(temp_list)
lists_test

[[[1, 1]], [[0, 1], [3, 1]], [[0, 1]], [[0, 1], [1, 1], [2, 1]]]

In [7]:
# 计算每个节点的出边和S_test
S_test = df['y'].value_counts()
S_test

3    3
1    2
2    1
0    1
Name: y, dtype: int64

In [8]:
# 另种意义上的转移概率矩阵 
for i in range(4):
    for j in range(len(lists_test[i])):
        if i in S_test.index:
            lists_test[i][j][1] = 1/S_test[i]
lists_test

[[[1, 1.0]],
 [[0, 0.5], [3, 0.5]],
 [[0, 1.0]],
 [[0, 0.3333333333333333], [1, 0.3333333333333333], [2, 0.3333333333333333]]]

In [9]:
# 计算importance
def page_rank_df(lists_test, threshold = 10e-3, beta = 0.8):
    N = 4
    r_old = np.array([0 for i in range(N)])
    r_new = np.array([(1/N) for i in range(N)])
    while np.sum(np.abs(r_old - r_new)) > threshold:
        r_old = r_new
        dot_result = []
        for i in range(N): #x
            if len(lists_test[i])==0:
                dot_result.append(0)
            else:
                temp_result = 0
                for j in range(len(lists_test[i])):
                    temp_result+=lists_test[i][j][1]*r_new[lists_test[i][j][0]]
                dot_result.append(temp_result)
            print('dot_result:', dot_result)
        r_new = beta * np.array(dot_result)
        print(r_new)
        S = np.sum(r_new)
        r_new += (1-S)/N
        print('after add', r_new)
    return r_new

In [10]:
page_rank_df(lists_test)

dot_result: [0.25]
dot_result: [0.25, 0.25]
dot_result: [0.25, 0.25, 0.25]
dot_result: [0.25, 0.25, 0.25, 0.25]
[0.2 0.2 0.2 0.2]
after add [0.25 0.25 0.25 0.25]


array([0.25, 0.25, 0.25, 0.25])

# Test Run

In [9]:
relationship = relationship.drop(':TYPE',axis = 1)
relationship = relationship.rename(columns={':START_ID':'x',':END_ID':'y'})
relationship

,x,y,inverse
0,362633,5533231,direct
1,362633,1056076,direct
2,362633,5348598,direct
3,362633,3773633,direct
4,1324976,3429596,direct
...,...,...,...
26377579,1862626,4326933,directed
26377580,1862626,193423,directed
26377581,5175169,4635056,directed
26377582,1862626,470273,directed


In [13]:
N = 5822932

In [72]:
# 另种意义上的邻接矩阵
start = time.time()
lists = []
for i in range(N):
    temp_series = relationship[relationship.x==i]
    if len(temp_series)==0:
        lists.append([])
    else:
        temp_list = []
        for j in range(len(temp_series)):
            temp_list.append([temp_series.y.values[j],1])
        lists.append(temp_list)
end = time.time()
print(end-start)
# lists
np.savez('lists',lists)
lists

92952.86974930763


[[[2391260, 1], [4524512, 1], [4068750, 1]],
 [[3959699, 1], [4718250, 1], [2814046, 1], [2374048, 1]],
 [[2987581, 1],
  [2987581, 1],
  [1783402, 1],
  [3162442, 1],
  [4026786, 1],
  [798334, 1]],
 [[1778489, 1],
  [1472713, 1],
  [3972974, 1],
  [3972974, 1],
  [3214570, 1],
  [5159616, 1],
  [4507324, 1]],
 [[1862626, 1]],
 [[679000, 1]],
 [],
 [],
 [],
 [[1003439, 1]],
 [],
 [[4838560, 1]],
 [[4478815, 1], [796431, 1], [799237, 1], [5763909, 1]],
 [[434585, 1], [1179445, 1], [1209987, 1]],
 [[2626316, 1], [68242, 1], [138887, 1]],
 [[1862626, 1]],
 [[2206374, 1],
  [2849982, 1],
  [5337052, 1],
  [4195976, 1],
  [1769816, 1],
  [2334245, 1],
  [5019919, 1],
  [1034022, 1],
  [4517581, 1],
  [3843046, 1],
  [3238440, 1]],
 [],
 [[3131546, 1], [4222879, 1], [4155678, 1], [235710, 1]],
 [[1862626, 1]],
 [],
 [[3192933, 1]],
 [[5457386, 1], [5457386, 1], [5670567, 1], [2082332, 1]],
 [],
 [[5774495, 1]],
 [[1739344, 1]],
 [[4435927, 1], [4947389, 1], [5006917, 1]],
 [[5507566, 1], [2

In [74]:
data = np.load('lists.npz', allow_pickle=True)
len(data['arr_0'].tolist())

5822932

In [75]:
# 计算每个节点的出边和S
S = relationship['y'].value_counts()
S

1862626    499618
5763909    236171
3238440    107685
4289275     89360
2774690     73555
            ...  
904808          1
884611          1
4908685         1
4851345         1
5551654         1
Name: y, Length: 2152483, dtype: int64

In [79]:
data = data['arr_0'].tolist()

In [80]:
# 另种意义上的转移概率矩阵 
start = time.time()
for i in range(N):
    for j in range(len(data[i])):
        if data[i][j][0] in S.index:
            data[i][j][1] = data[i][j][1]/S[data[i][j][0]]
end = time.time()
np.savez('TPM',data)
print(end-start)
data

171.46739053726196


[[[2391260, 0.0005279831045406547],
  [4524512, 0.05263157894736842],
  [4068750, 0.2]],
 [[3959699, 0.05263157894736842],
  [4718250, 0.002207505518763797],
  [2814046, 0.00016877637130801687],
  [2374048, 0.5]],
 [[2987581, 0.011627906976744186],
  [2987581, 0.011627906976744186],
  [1783402, 9.551098376313276e-05],
  [3162442, 0.0017094017094017094],
  [4026786, 0.5],
  [798334, 0.5]],
 [[1778489, 2.7927500209456252e-05],
  [1472713, 0.004347826086956522],
  [3972974, 0.02564102564102564],
  [3972974, 0.02564102564102564],
  [3214570, 0.00020016012810248197],
  [5159616, 0.00011270145384875464],
  [4507324, 0.00030349013657056146]],
 [[1862626, 2.0015291682845694e-06]],
 [[679000, 0.001288659793814433]],
 [],
 [],
 [],
 [[1003439, 0.058823529411764705]],
 [],
 [[4838560, 0.0017452006980802793]],
 [[4478815, 0.5],
  [796431, 9.521995810321844e-05],
  [799237, 0.015384615384615385],
  [5763909, 4.234220120167167e-06]],
 [[434585, 0.0004100041000410004],
  [1179445, 0.02173913043478260

In [9]:
tpm = np.load('TPM.npz', allow_pickle=True)
tpm = tpm['arr_0'].tolist()
print(len(tpm))

5822932


In [ ]:
tpm[]

In [123]:
# 计算importance
def page_rank_list(data, threshold = 1e-8, beta = 0.8):
    N = len(data)
    r_old = np.array([0 for i in range(N)])
    r_new = np.array([(1/N) for i in range(N)])
    e = 0
    while e < 10:
        r_old = r_new
        dot_result = []
        for i in range(N): #x
            if len(lists[i])==0:
                dot_result.append(0)
            else:
                temp_result = 0
                for j in range(len(lists[i])):
                    temp_result+=lists[i][j][1]*r_new[lists[i][j][0]]
                dot_result.append(temp_result)
        r_new = beta * np.array(dot_result)+(1-beta)*r_new
        e += 1
#         S = np.sum(r_new)
#         r_new += (1-S)/N
        print(r_new)
    return r_new

In [ ]:
s = time.time()
importance = page_rank_list(data)
np.savez('importance',importance)
e = time.time()
t = e-s
print(t)
importance

In [128]:
final = np.load('importance.npz', allow_pickle=True)
final = final['arr_0'].tolist()
print(len(final))
final

5822932


[8.734346035190878,
 10.663923884494352,
 2728.4533343003573,
 1.5197685116661102,
 0.16123125661283003,
 0.8410097410378637,
 1.7585642422064998e-14,
 1.7585642422064998e-14,
 1.7585642422064998e-14,
 3.123951287782855,
 1.7585642422064998e-14,
 0.4444729108247056,
 21.148010836510956,
 6.035221884781512,
 4.517939802866651,
 0.16123125661283003,
 25.507637293415858,
 1.7585642422064998e-14,
 13.437044064453296,
 0.16123125661283003,
 1.7585642422064998e-14,
 0.08844349758400367,
 5.9085012690964955,
 1.7585642422064998e-14,
 0.10685385462324204,
 0.9412252363562781,
 10.652950844749377,
 36.01930992523294,
 1.7585642422064998e-14,
 34.17194476518882,
 0.566531030827105,
 0.21873835193402977,
 1489.0570379837543,
 0.17496573686012593,
 7.789611756665836,
 0.16123125661283003,
 1.7585642422064998e-14,
 0.06558046826941806,
 44.30235630838357,
 1.7585642422064998e-14,
 1.7585642422064998e-14,
 53.99665050647782,
 1.7585642422064998e-14,
 4.107981590387703,
 1.7585642422064998e-14,
 1.55

# Revised Formal Run

In [18]:
relationship = relationship.drop(':TYPE',axis = 1)
relationship = relationship.rename(columns={':START_ID':'x',':END_ID':'y'})
relationship

,x,y,inverse
0,362633,5533231,direct
1,362633,1056076,direct
2,362633,5348598,direct
3,362633,3773633,direct
4,1324976,3429596,direct
...,...,...,...
26377579,1862626,4326933,directed
26377580,1862626,193423,directed
26377581,5175169,4635056,directed
26377582,1862626,470273,directed


In [30]:
relationship[relationship['y']==5822932]

,x,y,inverse
26160381,1862626,5822932,directed


In [25]:
N = 5822933

In [1]:
# 另种意义上的邻接矩阵
start = time.time()
lists = []
for i in range(N):  # demo test
    temp_series = relationship[relationship.y==i]
    if len(temp_series)==0:
        lists.append([])
    else:
        temp_list = []
        for j in range(len(temp_series)):
            temp_list.append([temp_series.x.values[j],1])
        lists.append(temp_list)
    if i%100 == 0:
        print(i)
end = time.time()
print(end-start)
# lists
np.savez('lists1',lists)
lists

In [101]:
data = np.load('lists1.npz', allow_pickle=True)
data = data['arr_0'].tolist()
print(len(data))

5822932


In [ ]:
data1 = np.load('lists1.npz', allow_pickle=True)
data1 = data1['arr_0'].tolist()
print(len(data1))

In [10]:
# 计算每个节点的出边和S
S = relationship['y'].value_counts()
S

1862626    499622
5763909    236183
3238440    107687
4289275     89360
2774690     73559
            ...  
3566408         1
3691440         1
3675064         1
3855328         1
46501           1
Name: y, Length: 4421150, dtype: int64

In [3]:
# 另种意义上的转移概率矩阵 
start = time.time()
for i in range(N):
    for j in range(len(data[i])):
        if data[i][j][0] in S.index:
            data[i][j][1] = data[i][j][1]/S[data[i][j][0]]
end = time.time()
np.savez('TPM1',data)
print(end-start)
data

In [4]:
tpm = np.load('TPM2.npz', allow_pickle=True)
tpm = tpm['arr_0'].tolist()
print(len(tpm))

5822932


In [5]:
tpm_or = np.load('TPM.npz', allow_pickle=True)
tpm_or = tpm_or['arr_0'].tolist()
print(len(tpm_or))

5822932


In [32]:
# 计算importance
def page_rank_list(data, threshold = 1e-8, beta = 0.8):
    N = len(data)
    r_old = np.array([0 for i in range(N)])
    r_new = np.array([(1/N) for i in range(N)])
    e = 0
    while e < 10:
        r_old = r_new
        dot_result = []
        for i in range(N): #x
            if len(data[i])==0:
                dot_result.append(0)
            else:
                temp_result = 0
                for j in range(len(data[i])):
                    if data[i][j][0] == N:
                        continue
                    temp_result+=data[i][j][1]*r_new[data[i][j][0]]
#                     print(temp_result)
                dot_result.append(temp_result)
#             print(i)
        r_new = beta * np.array(dot_result)+(1-beta)*r_new
        e += 1
#         S = np.sum(r_new)
#         r_new += (1-S)/N
        print(r_new)
    return r_new

In [13]:
tpm[1862626][340434]

[5822932, 1.0]

In [33]:
s = time.time()
importance = page_rank_list(tpm)
np.savez('importance',importance)
e = time.time()
t = e-s
print(t)
importance

[5.26076892e-08 1.63550088e-06 1.05655204e-07 ... 1.71895499e-07
 3.43472328e-08 4.96625240e-08]
[9.42026264e-08 9.22510927e-07 1.07153731e-07 ... 3.41137808e-07
 7.43442863e-08 7.02630890e-08]
[8.28847968e-08 1.12529533e-06 9.88487057e-08 ... 2.42378883e-07
 4.86706219e-08 6.78571341e-08]
[8.33977754e-08 9.81489557e-07 9.93941783e-08 ... 2.83175350e-07
 5.37251584e-08 6.91355040e-08]
[8.05180326e-08 9.88908608e-07 9.96653564e-08 ... 2.58215091e-07
 4.60239405e-08 6.98041985e-08]
[8.04377789e-08 9.47009031e-07 1.02024199e-07 ... 2.66399170e-07
 4.51434011e-08 7.01743959e-08]
[8.01531691e-08 9.30400809e-07 1.04725408e-07 ... 2.58948335e-07
 4.17631069e-08 7.04742742e-08]
[8.06387959e-08 9.09627867e-07 1.07989949e-07 ... 2.59618084e-07
 4.01722182e-08 7.06019167e-08]
[8.11444482e-08 8.93924204e-07 1.11393924e-07 ... 2.56648662e-07
 3.82191503e-08 7.06716687e-08]
[8.18440539e-08 8.79629603e-07 1.14916046e-07 ... 2.55784625e-07
 3.68582179e-08 7.06668374e-08]
676.6678521633148


array([8.18440539e-08, 8.79629603e-07, 1.14916046e-07, ...,
       2.55784625e-07, 3.68582179e-08, 7.06668374e-08])

In [ ]:
m = 0
for i in range(len(data)):
    for j in range(len(data[i])):
        if data[i][j][0]>m:
            m = data[i][j][0]

In [ ]:
final = np.load('importance.npz', allow_pickle=True)
final = final['arr_0'].tolist()
print(len(final))
final